Upload file Kaggle.json untukmengambil file di kaggle.com

In [1]:
# from google.colab import files
# files.upload()

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [3]:
!kaggle datasets download clmentbisaillon/fake-and-real-news-dataset

fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!ls

data  fake-and-real-news-dataset.zip  kaggle.json  sample_data


Extract data dari file zip

In [5]:
import pandas as pd
import zipfile
zip_ref = zipfile.ZipFile('fake-and-real-news-dataset.zip', 'r')
zip_ref.extractall('data')
zip_ref.close()

In [6]:
import pandas as pd
df_fake = pd.read_csv('/content/data/Fake.csv')
df_true = pd.read_csv('/content/data/True.csv')
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


Menambah label fake/real sesuai dengan jenis berita

In [7]:
df_true['label'] = 'real'
df_true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",real


In [8]:
df_fake['label'] = 'fake'
df_fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",fake


Menggabungkan dataframe berita Real dan Fake

In [9]:
df_news = pd.concat([df_true, df_fake])
df_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",real


Proses One-Hot_Encoding

In [10]:
category = pd.get_dummies(df_news.label)
df_new_news = pd.concat([df_news, category], axis=1)
df_new_news = df_new_news.drop(columns='label')
df_new_news.head()

,title,text,subject,date,fake,real
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,1


Hapus column tidak penting

In [11]:
df_new_news = df_new_news.drop(columns=['title', 'subject', 'date'])
df_new_news.tail()

,text,fake,real
23476,21st Century Wire says As 21WIRE reported earl...,1,0
23477,21st Century Wire says It s a familiar theme. ...,1,0
23478,Patrick Henningsen 21st Century WireRemember ...,1,0
23479,21st Century Wire says Al Jazeera America will...,1,0
23480,21st Century Wire says As 21WIRE predicted in ...,1,0


In [12]:
df_new_news

,text,fake,real
0,WASHINGTON (Reuters) - The head of a conservat...,0,1
1,WASHINGTON (Reuters) - Transgender people will...,0,1
2,WASHINGTON (Reuters) - The special counsel inv...,0,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0,1
...,...,...,...
23476,21st Century Wire says As 21WIRE reported earl...,1,0
23477,21st Century Wire says It s a familiar theme. ...,1,0
23478,Patrick Henningsen 21st Century WireRemember ...,1,0
23479,21st Century Wire says Al Jazeera America will...,1,0


Hapus row yang memiliki nilai null

In [13]:
df = df_new_news.dropna()
df.isnull().sum()

text    0
fake    0
real    0
dtype: int64

**case folding**

In [14]:
df["text"] = df["text"].str.lower()
df.head()

,text,fake,real
0,washington (reuters) - the head of a conservat...,0,1
1,washington (reuters) - transgender people will...,0,1
2,washington (reuters) - the special counsel inv...,0,1
3,washington (reuters) - trump campaign adviser ...,0,1
4,seattle/washington (reuters) - president donal...,0,1


**Punctuation Removal**

In [15]:
df["text"] = df['text'].str.replace('[^\w\s]','')
df.head()

,text,fake,real
0,washington reuters the head of a conservative...,0,1
1,washington reuters transgender people will be...,0,1
2,washington reuters the special counsel invest...,0,1
3,washington reuters trump campaign adviser geo...,0,1
4,seattlewashington reuters president donald tr...,0,1


Stopwords Removal

In [16]:
!python -m nltk.downloader stopwords

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,text,fake,real
0,washington reuters head conservative republica...,0,1
1,washington reuters transgender people allowed ...,0,1
2,washington reuters special counsel investigati...,0,1
3,washington reuters trump campaign adviser geor...,0,1
4,seattlewashington reuters president donald tru...,0,1


In [18]:
text = df['text'].values
label = df[['fake', 'real']].values

Split data menjadi data train dan test dengan rasio 8:2

In [19]:
from sklearn.model_selection import train_test_split
text_train, text_test, label_train, label_test = train_test_split(text, label, test_size=0.2)

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

Tokenizing

In [21]:
tokenizer = Tokenizer(num_words=5000, 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, 
                      oov_token='x')
tokenizer.fit_on_texts(text_train)
tokenizer.fit_on_texts(text_test)

sequences_train = tokenizer.texts_to_sequences(text_train)
sequences_test = tokenizer.texts_to_sequences(text_test)

padded_train = pad_sequences(sequences_train)
padded_test = pad_sequences(sequences_test)

Pembuatan Model

In [22]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=6000, output_dim=16),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
class stopCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi telah mencapai 90%!")
      self.model.stop_training = False
    if(logs.get('val_accuracy')>0.9):
      print("\nAkurasi data validasi telah mencapai 90%!")
      self.model.stop_training = True
callbacks = stopCallback()

Train Model

In [24]:
num_epochs = 10
history = model.fit(padded_train, label_train, epochs=num_epochs, batch_size=128,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)

plt.show()

In [ ]:
import numpy as np

txt = {'text_predict':["Today, President Joe Biden announced two additional members of his energy and jobs team: David Turk, Deputy Secretary of Energy and Julie Su, Deputy Secretary of Labor. These tested and experienced leaders will support the work of Secretary of Energy Nominee Jennifer Granholm and Secretary of Labor Nominee Marty Walsh in their efforts to address the worst energy and jobs crisis in nearly a century.\
Turk and Su will help advance President Biden’s Build Back Better agenda by rebuilding America’s middle class and creating an equitable clean energy future we can depend on.  They will be partners to the President in building a stronger, more resilient, and more inclusive economy that delivers every American a fair return for their work and an equal chance to get ahead.\
The following White House announcements were made today: \
David Turk, Deputy Secretary of Energy\
Dave Turk is currently the Deputy Executive Director of the International Energy Agency (IEA). Throughout his tenure at the IEA, Turk has focused on helping countries around the world on their clean energy transitions. He has also directed analysis focused on digitalization and energy, hydrogen, and tracking progress on a wide range of clean energy technologies.\
Turk also served as the Deputy on the Energy Agency Review Team during the transition to the Biden-Harris Administration, which provided recommendations across the full range of Department of Energy (DOE) issues and offices.\
During the Obama-Biden Administration, Turk worked in DOE coordinating international technology and clean energy efforts. During this time, he helped spearhead the launch of Mission Innovation, a global effort to enhance clean energy innovation.\
Turk has also served as Special Assistant to the President and Senior Director at the U.S. National Security Council, where he coordinated interagency legislative affairs efforts by the full range of national security Agencies and provided legislative advice to National Security Council decision-making. He also previously worked at the U.S. Department of State, including as Deputy Special Envoy for Climate Change and helping to coordinate New Start Treaty ratification efforts in the U.S. Senate.  \
Earlier in his career, Turk worked in both the U.S. Senate, primarily on national security issues, and as the Staff Director of the National Security Subcommittee of the House Oversight Committee.\
Turk was born in Quito, Ecuador, and grew up in a small Midwestern town. He is married to Emily Turk, a registered architect and sustainability professional, and they have three children.\
Julie Su, Deputy Secretary of Labor\
Julie A. Su is currently the Secretary of the California Labor and Workforce Development Agency. Governor Gavin Newsom appointed Su in January of 2019 to serve as his cabinet advisor on labor issues and employment programs for workers and businesses throughout California.\
Secretary Su oversees the state departments and boards that enforce labor laws, including minimum wage and occupational safety standards, provide state disability and unemployment insurance benefits, fund workforce training and apprenticeship programs, combat wage theft, protect injured workers, and arbitrate public sector contract disputes.\
Su is a nationally recognized expert on workers’ rights and civil rights who has dedicated her distinguished legal career to advancing justice on behalf of poor and disenfranchised communities, and is a past recipient of a MacArthur Foundation “Genius” grant.\
As California Labor Commissioner from 2011 through 2018, Su enforced the state’s labor laws to ensure a fair and just workplace for both employees and employers. A report on her tenure released in May 2013 found that her leadership has resulted in a renaissance in enforcement activity and record-setting results. In 2014, she launched the first “Wage Theft is a Crime” multi-media, multilingual statewide campaign to reach out to low-wage workers and their employers to help them understand their rights and feel safe speaking up about labor law abuses. \
Prior to her appointment as California Labor Commissioner, Su was the Litigation Director at Asian Americans Advancing Justice-Los Angeles, the nation’s largest non-profit civil rights organization devoted to issues affecting the Asian American community. In her 17 years as a civil rights lawyer, Su brought landmark lawsuits resulting in millions of dollars for low-wage workers and policy changes in California and the United States protecting immigrant victims of crime and human trafficking. In 1995, she was the lead attorney for Thai garment workers who were trafficked into the United States and forced to sew behind barbed wire and under armed guard in an apartment complex in El Monte, California. Su is known for pioneering a multi-strategy approach that combines successful impact litigation with multiracial organizing, community education, policy reform, coalition building, and media work.\
Su has taught at UCLA Law School and Northeastern Law School. She is a graduate of Stanford University and Harvard Law School and began her career with a Skadden Fellowship. Su is the daughter of Chinese immigrants and speaks Mandarin and Spanish."]}

dfPredict = pd.DataFrame (txt, columns = ['text_predict'])

stopPredict = stopwords.words('english')
dfPredict["text_predict"] = dfPredict["text_predict"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

seq = tokenizer.texts_to_sequences(dfPredict["text_predict"])
padded = pad_sequences(seq)
pred = model.predict(padded)
labels = ['Fake News', 'Real News']

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)])

In [ ]:
import numpy as np

txt = {'text_predict':["Print They should pay all the back all the money plus interest. The entire family and everyone who came in with them need to be deported asap. Why did it take two years to bust them? Here we go again …another group stealing from the government and taxpayers! A group of Somalis stole over four million in government benefits over just 10 months! We’ve reported on numerous cases like this one where the Muslim refugees/immigrants commit fraud by scamming our system…It’s way out of control! More Related"]}

dfPredict = pd.DataFrame (txt, columns = ['text_predict'])

stopPredict = stopwords.words('english')
dfPredict["text_predict"] = dfPredict["text_predict"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

seq = tokenizer.texts_to_sequences(dfPredict["text_predict"])
padded = pad_sequences(seq)
pred = model.predict(padded)
labels = ['Fake News', 'Real News']

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)])